# Data Description

# Tasks to be performed

In [22]:
# Importing necessary libraries for data interpretation and training the model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [23]:
df = pd.read_csv("data_travit.csv")

In [24]:
df.head(10)

,timestamp,part_id,temperature_1,temperature_2,temperature_3,temperature_4,label
0,2024-02-15 05:44:34+05:30,2124077390649042,439.1,424.1,536.0,518.6,0
1,2024-02-15 05:44:35+05:30,2124077390649042,438.9,423.9,535.9,518.7,0
2,2024-02-15 05:44:36+05:30,2124077390649042,438.7,423.6,535.6,518.9,0
3,2024-02-15 05:44:37+05:30,2124077390649042,438.4,423.4,535.4,519.0,0
4,2024-02-15 05:44:38+05:30,2124077390649042,438.2,423.2,535.2,519.2,0
5,2024-02-15 05:44:39+05:30,2124077390649042,438.1,423.0,535.0,519.2,0
6,2024-02-15 05:44:40+05:30,2124077390649042,437.9,422.7,534.7,519.4,0
7,2024-02-15 05:44:41+05:30,2124077390649042,437.7,422.5,534.6,519.5,0
8,2024-02-15 05:44:42+05:30,2124077390649042,437.5,422.3,534.4,519.6,0
9,2024-02-15 05:44:43+05:30,2124077390649042,437.4,422.1,534.2,519.7,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   timestamp      100000 non-null  object 
 1   part_id        100000 non-null  int64  
 2   temperature_1  100000 non-null  float64
 3   temperature_2  100000 non-null  float64
 4   temperature_3  100000 non-null  float64
 5   temperature_4  100000 non-null  float64
 6   label          100000 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 5.3+ MB


In [26]:
df.describe()

,part_id,temperature_1,temperature_2,temperature_3,temperature_4,label
count,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2.124078e+15,455.534524,440.114258,546.868253,517.654658,0.059700
std,2.901028e+09,20.596332,23.310541,24.818341,17.513545,0.236931
min,2.124077e+15,350.800000,329.600000,407.700000,435.600000,0.000000
25%,2.124077e+15,440.600000,425.200000,532.600000,508.500000,0.000000
50%,2.124077e+15,454.000000,441.300000,550.000000,518.300000,0.000000
75%,2.124077e+15,472.600000,459.500000,567.200000,530.200000,0.000000
max,2.124088e+15,501.200000,481.000000,582.700000,555.400000,1.000000


In [27]:
# setting the parameters
k = 5 #window size

# Create the dataset for predictive modeling
def create_windows(df, k):
    windows = []
    labels = []
    grouped = df.groupby('part_id')
    # collect all parts in a list
    parts = [group for _, group in grouped]

    for i in range(len(parts) - k +1):
        window = pd.concat(parts[i:i+k])
        label = parts[i+k-1]['label'].values[0]
        windows.append(window)
        labels.append(label)
    return windows, labels

windows , labels = create_windows(df, k)


# Feature Engineering

def extract_features(window):
    features = {}
    for col in ['temperature_1', 'temperature_2', 'temperature_3', 'temperature_4']:
        features[f'{col}_max'] = window[col].max()
        features[f'{col}_min'] = window[col].min()
        features[f'{col}_std'] = window[col].std()
    return features

feature_list = [extract_features(window) for window in windows]
features_df = pd.DataFrame(feature_list)

# Train a predictive model
x = features_df
y = labels

# split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

# initialize and train the model
model = RandomForestClassifier(random_state = 42)
model.fit(x_train, y_train)

# Predictions and Evaluations
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


        

Accuracy: 0.9152542372881356
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        54
           1       0.00      0.00      0.00         5

    accuracy                           0.92        59
   macro avg       0.46      0.50      0.48        59
weighted avg       0.84      0.92      0.87        59

